<a href="https://colab.research.google.com/github/cormach/bayesian_stats_by_b_lambert/blob/master/Chapter_16_q_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import shutil
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Please use the latest version of CmdStanPy
!pip install --upgrade cmdstanpy

ERROR: fbprophet 0.6 has requirement cmdstanpy==0.4, but you'll have cmdstanpy 0.9.5 which is incompatible.
  Found existing installation: cmdstanpy 0.4.0
    Uninstalling cmdstanpy-0.4.0:
      Successfully uninstalled cmdstanpy-0.4.0


In [3]:
# Install pre-built CmdStan binary
# (faster than compiling from source via install_cmdstan() function)
tgz_file = 'colab-cmdstan-2.23.0.tar.gz'
tgz_url = 'https://github.com/stan-dev/cmdstan/releases/download/v2.23.0/colab-cmdstan-2.23.0.tar.gz'
if not os.path.exists(tgz_file):
    urllib.request.urlretrieve(tgz_url, tgz_file)
    shutil.unpack_archive(tgz_file)

In [4]:
# Specify CmdStan location via environment variable
os.environ['CMDSTAN'] = './cmdstan-2.23.0'
# Check CmdStan path
from cmdstanpy import CmdStanModel, cmdstan_path
cmdstan_path()

'./cmdstan-2.23.0'

In [5]:
!pip install arviz

     |████████████████████████████████| 1.5MB 4.6MB/s 
     |████████████████████████████████| 4.3MB 20.6MB/s 
     |████████████████████████████████| 296kB 43.4MB/s 


In [6]:
import arviz as az

Question 16_2 - hangover

In [38]:
stan_text = '''data {
    int<lower=1> nStudy;
    int<lower=1> N;
    int<lower=0, upper=N> X[nStudy];
}
parameters {
    ordered[2] alpha;
}
transformed parameters{
  real<lower=0,upper=1> theta[2];
  matrix[nStudy,2] lp;
  for(i in 1:2)
    theta[i] = inv_logit(alpha[i]);

  for(n in 1:nStudy)
    for(s in 1:2)
      lp[n,s] = log(0.5) + binomial_logit_lpmf(X[n]| N, alpha[s]);
}
model {
  for(n in 1:nStudy)
    target += log_sum_exp(lp[n]);
}
generated quantities{
  matrix[nStudy, 2] pstate;
  for(n in 1: nStudy)
   pstate[n] = exp(lp[n] - log_sum_exp(lp[n]));
}
'''
with open('stan_file.stan', 'w') as f:
  f.write(stan_text)

In [39]:
!cat stan_file.stan

data {
    int<lower=1> nStudy;
    int<lower=1> N;
    int<lower=0, upper=N> X[nStudy];
}
parameters {
    ordered[2] alpha;
}
transformed parameters{
  real<lower=0,upper=1> theta[2];
  matrix[nStudy,2] lp;
  for(i in 1:2)
    theta[i] = inv_logit(alpha[i]);

  for(n in 1:nStudy)
    for(s in 1:2)
      lp[n,s] = log(0.5) + binomial_logit_lpmf(X[n]| N, alpha[s]);
}
model {
  for(n in 1:nStudy)
    target += log_sum_exp(lp[n]);
}
generated quantities{
  matrix[nStudy, 2] pstate;
  for(n in 1: nStudy)
   pstate[n] = exp(lp[n] - log_sum_exp(lp[n]));
}


In [40]:
stan_model = CmdStanModel(stan_file='stan_file.stan')

INFO:cmdstanpy:compiling stan program, exe file: /content/stan_file
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /content/stan_file


In [34]:
data = {'nStudy':10, 'N':20,'X':[4,18,6,4,5,6,4,6,16,7]}

In [41]:
stan_posterior=stan_model.sample(data=data)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 4


In [42]:
stan_posterior.diagnose()

INFO:cmdstanpy:Processing csv files: /tmp/tmpo2vbbaic/stan_file-202007261623-1-jlq_wdxt.csv, /tmp/tmpo2vbbaic/stan_file-202007261623-2-ax775st5.csv, /tmp/tmpo2vbbaic/stan_file-202007261623-3-5bqcvvgb.csv, /tmp/tmpo2vbbaic/stan_file-202007261623-4-buqhucj4.csv

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory for all transitions.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete, no problems detected.


'Processing csv files: /tmp/tmpo2vbbaic/stan_file-202007261623-1-jlq_wdxt.csv, /tmp/tmpo2vbbaic/stan_file-202007261623-2-ax775st5.csv, /tmp/tmpo2vbbaic/stan_file-202007261623-3-5bqcvvgb.csv, /tmp/tmpo2vbbaic/stan_file-202007261623-4-buqhucj4.csv\n\nChecking sampler transitions treedepth.\nTreedepth satisfactory for all transitions.\n\nChecking sampler transitions for divergences.\nNo divergent transitions found.\n\nChecking E-BFMI - sampler transitions HMC potential energy.\nE-BFMI satisfactory for all transitions.\n\nEffective sample size satisfactory.\n\nSplit R-hat values satisfactory all parameters.\n\nProcessing complete, no problems detected.'

In [43]:
stan_posterior.summary().round(decimals=3)

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-24.133,0.024,1.027,-26.106,-23.818,-23.174,1800.37,5228.22,1.001
alpha[1],-1.046,0.004,0.182,-1.338,-1.047,-0.742,2162.05,6278.53,0.999
alpha[2],1.817,0.008,0.454,1.096,1.796,2.605,2882.33,8370.19,1.000
theta[1],0.262,0.001,0.035,0.208,0.260,0.323,2159.68,6271.66,0.999
theta[2],0.852,0.001,0.055,0.750,0.858,0.931,2607.25,7571.38,1.000
"lp[1,1]",-2.477,0.005,0.245,-2.964,-2.412,-2.224,2148.24,6238.43,1.000
"lp[1,2]",-24.531,0.110,5.989,-35.316,-23.996,-15.530,2953.84,8577.88,1.000
"lp[2,1]",-20.357,0.050,2.323,-24.195,-20.309,-16.594,2164.89,6286.79,0.999
"lp[2,2]",-2.337,0.011,0.512,-3.404,-2.140,-1.950,2019.24,5863.82,1.001
